# 中文自然语言处理分析

和拉丁语系不同，亚洲语言是不用空格分开每个有意义的词的。而当我们进行自然语言处理的时候，大部分情况下，词汇是我们对句子和文章理解的基础，因此需要一个工具去把完整的文本中分解成粒度更细的词。

 ## 1.关键词提取

### 基于 TF-IDF 算法的关键词抽取

import jieba.analyse

* jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())  
  * sentence 为待提取的文本  
  * topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20  
  * withWeight 为是否一并返回关键词权重值，默认值为 False  
  * allowPOS 仅包括指定词性的词，默认值为空，即不筛选

In [2]:
import jieba.analyse as analyse
import pandas as pd
df = pd.read_csv("./data/technology_news.csv", encoding='utf-8')  # 读取科技新闻
df = df.dropna()  # 老规矩，去掉缺省数据
lines=df.content.values.tolist()  # 转化为list形式
content = "".join(lines)  # 将序列中的元素以指定的字符连接生成一个新的字符串
print("  ".join(analyse.extract_tags(content, topK=30, withWeight=False, allowPOS=())))  # 抽取出30个权重最高的词进行打印

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/1q/57zhs1tx0230_c9b_0hc055m0000gn/T/jieba.cache
Loading model cost 0.795 seconds.
Prefix dict has been built succesfully.


用户  2016  互联网  手机  平台  人工智能  百度  2017  智能  技术  数据  360  服务  直播  产品  企业  安全  视频  移动  应用  网络  行业  游戏  机器人  电商  内容  中国  领域  通过  发展


In [3]:
import jieba.analyse as analyse
import pandas as pd
df = pd.read_csv("./data/military_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()
content = "".join(lines)
print("  ".join(analyse.extract_tags(content, topK=30, withWeight=False, allowPOS=())))  # analyse.extract_tags的运用

航母  训练  海军  中国  官兵  部队  编队  作战  10  任务  美国  导弹  能力  20  2016  军事  无人机  装备  进行  记者  我们  军队  安全  保障  12  战略  军人  日本  南海  战机


### 基于 TextRank 算法的关键词抽取

* jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')) 直接使用，接口相同，注意默认过滤词性。
* jieba.analyse.TextRank() 新建自定义 TextRank 实例

算法论文：[TextRank: Bringing Order into Texts](http://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf "TextRank: Bringing Order into Texts")

基本思想:

* 将待抽取关键词的文本进行分词
* 以固定窗口大小(默认为5，通过span属性调整)，词之间的共现关系，构建图
* 计算图中节点的PageRank，注意是无向带权图

In [4]:
import jieba.analyse as analyse
import pandas as pd
df = pd.read_csv("./data/military_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()
content = "".join(lines)

print("  ".join(analyse.textrank(content, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))))
print("---------------------我是分割线----------------")  # analyse.textrank（其余与之前一样）
print("  ".join(analyse.textrank(content, topK=20, withWeight=False, allowPOS=('ns', 'n'))))

中国  海军  训练  美国  部队  进行  官兵  航母  作战  任务  能力  军事  发展  工作  国家  问题  建设  导弹  编队  记者
---------------------我是分割线----------------
中国  海军  美国  部队  官兵  航母  军事  国家  任务  能力  导弹  技术  问题  日本  军队  编队  装备  系统  记者  战略


## 2.LDA主题模型

咱们来用LDA主题模型建模，看看这些新闻主要在说哪些topic。

首先我们要把文本内容处理成固定的格式，一个包含句子的list，list中每个元素是分词后的词list。类似下面这个样子。

[[第，一，条，新闻，在，这里],[第，二，条，新闻，在，这里],[这，是，在，做， 什么],...]

In [20]:
from gensim import corpora, models, similarities
import gensim

### 载入停用词

In [21]:
stopwords=pd.read_csv("data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

### 转换成合适的格式

In [66]:
import jieba
import pandas as pd
df = pd.read_csv("./data/technology_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()

sentences=[]  # 存储了分词之后的词组
for line in lines:
    try:
        segs=jieba.lcut(line)
        # 过滤掉停用词、空值以及换行符
        segs = filter(lambda x:len(x)>1, segs)  
        # 注意此时如果不将其强制转化为list的话其格式为filter，不能进行下一步操作
        segs = list(filter(lambda x:x not in stopwords, segs))  
        sentences.append(segs)
    except Exception as e:
        print(line)
        continue

In [70]:
# print(type(sentences))
for word in sentences[5]:
    print(word)

本次
商汤
带来
黄仁勋
展示
遥相呼应
SenseFace
人脸
布控
系统
千万级
人员
库中
300ms
识别
瞬间
锁定目标
功耗
十几
当属
人脸
布控
一大
科技


### 词袋模型

In [73]:
dictionary = corpora.Dictionary(sentences)  # 创建词袋模型
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]  # 利用doc2bow对其进行分割

In [74]:
corpus[5]

[(21, 1),
 (39, 1),
 (61, 1),
 (68, 1),
 (78, 1),
 (82, 1),
 (91, 1),
 (92, 1),
 (103, 1),
 (104, 2),
 (105, 2),
 (124, 1),
 (129, 1),
 (130, 1),
 (131, 1),
 (132, 1),
 (133, 1),
 (134, 1),
 (135, 1),
 (136, 1),
 (137, 1),
 (138, 1)]

### LDA建模

In [75]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

我们查一下第3号分类，其中最常出现的单词是：

我们把所有的主题打印出来看看

In [76]:
for topic in lda.print_topics(num_topics=20, num_words=8):
    print(topic[1])

0.030*"数据" + 0.029*"用户" + 0.018*"360" + 0.015*"攻击" + 0.014*"系统" + 0.013*"漏洞" + 0.011*"服务" + 0.010*"信息"
0.026*"城市" + 0.012*"北京" + 0.009*"社交" + 0.009*"老师" + 0.008*"上海" + 0.008*"医疗" + 0.008*"母婴" + 0.006*"医生"
0.025*"手机" + 0.014*"支付" + 0.013*"用户" + 0.010*"电脑" + 0.007*"短信" + 0.006*"电话" + 0.006*"隐私" + 0.006*"信息"
0.012*"流量" + 0.009*"媒体" + 0.008*"大学生" + 0.007*"大学" + 0.007*"学院" + 0.007*"科研" + 0.007*"阅读" + 0.006*"文娱"
0.024*"公司" + 0.022*"亿元" + 0.020*"网络" + 0.013*"诈骗" + 0.013*"小米" + 0.009*"乐视" + 0.008*"万元" + 0.008*"电信"
0.011*"黑客" + 0.009*"文件" + 0.009*"利用" + 0.008*"网络" + 0.007*"技术" + 0.007*"一带" + 0.007*"套餐" + 0.006*"一路"
0.021*"病毒" + 0.014*"驾驶" + 0.014*"汽车" + 0.013*"自动" + 0.012*"设计" + 0.012*"技术" + 0.011*"识别" + 0.009*"智能"
0.028*"服务" + 0.026*"智能" + 0.013*"平台" + 0.013*"工作" + 0.012*"企业" + 0.012*"生活" + 0.011*"互联网" + 0.010*"办公"
0.009*"信息" + 0.009*"防御" + 0.008*"委员会" + 0.007*"手机" + 0.007*"数据" + 0.007*"输入法" + 0.006*"样本" + 0.006*"用户"
0.062*"游戏" + 0.034*"用户" + 0.024*"手机" + 0.012*"报告" + 0.011*"玩家" + 0.011*"软件" +

我们可以对新加入的文本，进行简单主题分类：

lda.get_document_topics(bow)